# Modelo 1: Resnet50V2


In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

## Carga de datos

1. Denifinir los directorios

In [3]:
train_dir = "/Users/inma/Desktop/tfg/datos/train"
test_dir = "/Users/inma/Desktop/tfg/datos/test"
val_dir = "/Users/inma/Desktop/tfg/datos/valid"

2. Establecer los generadores de imágenes

In [4]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255) # rescale = reducir la dimension
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

3. Extraer las imágenes de los directorios

In [5]:
train_data = train_gen.flow_from_directory(train_dir,
                                           target_size = (224,224), # nueva escalacion
                                           class_mode = "categorical", # la clase es categorica: melanoma
                                           batch_size = 64 # tamaño del lote 
                                          ) 
test_data = test_gen.flow_from_directory(test_dir,
                                           target_size = (224,224),
                                           class_mode = "categorical",
                                           batch_size = 64
                                          )
val_data = val_gen.flow_from_directory(val_dir,
                                           target_size = (224,224),
                                           class_mode = "categorical",
                                           batch_size = 64
                                          )

Found 2000 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


## Modelo

1. Cargar el Resnet50V2

In [6]:
# building a resnet model
resnet_base = tf.keras.applications.ResNet50V2(include_top= False) # hay muchos modelos

resnet_base.trainable = False # para que no entrene lo ya pre-entrenado

2022-03-18 13:41:56.517183: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2. Crear el modelo

In [7]:
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(224,224,3)) # puedo cambiar el tamaño???

x = resnet_base(inputs)
x = layers.GlobalAveragePooling2D()(x) # capa de pooling
x = layers.Dense(128, activation = "relu")(x) # capa muy conectada, con argumento de activacion
# x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation = "relu")(x) # se repite -> mirar teoría
outputs = layers.Dense(3, activation="softmax")(x)

resnet_model = tf.keras.Model(inputs,outputs) # modelo

3. Compilar el modelo

In [8]:
resnet_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ["accuracy"]
)

4. Entrenamiento y validación

In [9]:
resnet_model.fit(train_data,
                 epochs = 5,
                 steps_per_epoch = len(train_data),
                 validation_data = val_data,
                 validation_steps = len(val_data)
                 )

Epoch 1/5
32/32 [==============================] - 336s 10s/step - loss: 0.7688 - accuracy: 0.6840 - val_loss: 0.8174 - val_accuracy: 0.6200
Epoch 2/5
32/32 [==============================] - 317s 10s/step - loss: 0.5728 - accuracy: 0.7655 - val_loss: 0.8596 - val_accuracy: 0.6533
Epoch 3/5
32/32 [==============================] - 2000s 64s/step - loss: 0.5179 - accuracy: 0.7845 - val_loss: 0.6882 - val_accuracy: 0.7333
Epoch 4/5
32/32 [==============================] - 311s 10s/step - loss: 0.4385 - accuracy: 0.8230 - val_loss: 0.7202 - val_accuracy: 0.6933
Epoch 5/5
32/32 [==============================] - 316s 10s/step - loss: 0.3757 - accuracy: 0.8480 - val_loss: 0.7636 - val_accuracy: 0.6867


In [10]:
resnet_base.trainable = True

for layer in resnet_base.layers[:-5]:
  resnet_base.trainable = False

resnet_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

5. Entrenamiento sobre el modelo tuneado

In [11]:
resnet_model.fit(train_data,
                 epochs = 5,
                 steps_per_epoch = len(train_data),
                 validation_data = val_data,
                 validation_steps = len(val_data)
                 )

Epoch 1/5
32/32 [==============================] - 325s 10s/step - loss: 0.3742 - accuracy: 0.8480 - val_loss: 0.9479 - val_accuracy: 0.7000
Epoch 2/5
32/32 [==============================] - 312s 10s/step - loss: 0.2872 - accuracy: 0.8930 - val_loss: 1.0824 - val_accuracy: 0.6600
Epoch 3/5
32/32 [==============================] - 312s 10s/step - loss: 0.2409 - accuracy: 0.9250 - val_loss: 0.9204 - val_accuracy: 0.7000
Epoch 4/5
32/32 [==============================] - 327s 10s/step - loss: 0.1764 - accuracy: 0.9495 - val_loss: 0.9881 - val_accuracy: 0.7000
Epoch 5/5
32/32 [==============================] - 328s 10s/step - loss: 0.1472 - accuracy: 0.9575 - val_loss: 0.9382 - val_accuracy: 0.7067


## Guardar el modelo

In [12]:
resnet_model.save(
    'modelos/resnet_model',
    overwrite=True,
)

2022-03-18 15:03:46.154361: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: modelos/resnet_model/assets
